In [9]:
from transformers import AutoTokenizer, AutoModel
import json 
from tqdm import tqdm

device = 'cuda:2'
chatglm_path = "chatglm3-6b/chatglm3-6b"
chatglm = AutoModel.from_pretrained(chatglm_path, trust_remote_code=True, device=device)
chatglm = chatglm.eval()



Loading checkpoint shards: 100%|██████████| 7/7 [00:10<00:00,  1.43s/it]


In [28]:
def get_answer(data,prompt_template,chatglm,params,
               loop = True) -> str:
    """
    通过 ChatGLM 进行QA
    """

    inputs = {
        "question": data["question"],
        "info":data["related_str"]
        }

    tokenizer = AutoTokenizer.from_pretrained(chatglm_path, trust_remote_code=True)
    # Execute the chain
    response0, history = chatglm.chat(tokenizer,prompt_template[0].format(inputs["question"],inputs["info"]), history=[],**params)
    # print(prompt_template[0].format(inputs["question"],inputs["info"][0] + inputs["info"][1]))
    if loop:
        response, history = chatglm.chat(tokenizer,prompt_template[1].format(inputs["question"]), history=history,**params)

    return [response0,response]


In [29]:
with open("result/related_str.json", "r", encoding="utf-8") as file:
    json_data = file.read()
datas = json.loads(json_data)

In [31]:
prompt_template = ["""你是一位智能汽车使用说明的问答助手，现在需要根据已有信息，完整简要地回答问题，注意A在B内，B在C内，不要推断A在C内。问题是：{}\n已有信息是：{}""",
    """请简要地总结先前的回答，只保留与问题最相关的部分，在总结中不要重复问题。问题是：{}"""]

max_length = 4096
top_p      = 0.6
temperature= 0.5
params = {"max_length":max_length,"top_p":top_p,"temperature":temperature}

results = []
for data in tqdm(datas,desc="question"):
    ret = get_answer(data,prompt_template,chatglm,params)
    sample = {"question": data["question"], "answer_1": ret[0], "answer_2": ret[1],"answer_3":""}
    results.append(sample)
    # print(sample["answer_1"])
    # print("---")
    # print(sample["answer_2"])



question: 100%|██████████| 100/100 [10:34<00:00,  6.35s/it]


In [ ]:
import json

# 读取JSON文件
input_file = 'result/74.20.json'
output_file = 'result/final10.json'

with open(input_file, 'r', encoding="utf-8") as file:
    data = json.load(file)

# 保留question和answer字段
filtered_data = []
for item,result in zip(data,results):
    filtered_item = {
        "question": item.get("question"),
        "answer_1": result.get("answer_1").replace("\n", ""),
        "answer_2": result.get("answer_2").replace("\n", ""),
        "answer_3": item.get("answer_3").replace("\n", "")
    }
    filtered_data.append(filtered_item)

# 写入新的JSON文件
with open(output_file, 'w', encoding="utf-8") as file:
    json.dump(filtered_data, file,ensure_ascii=False,indent=4)

In [32]:
with open('result/final10.json', "w", encoding="utf-8") as file :
    json.dump(results, file,ensure_ascii=False,indent=4)